In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time

In [2]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 1800
eval_interval = 10
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 100
n_embd = 512
n_head = 8
n_layer = 8
dropout = 0.0
print(device)
# ------------

cuda


In [3]:
torch.manual_seed(1337)

with open("/content/drive/MyDrive/Colab Notebooks/shayri_dataset.txt", 'r', encoding='utf-8') as f:
    text = f.read()
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [4]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [5]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.99*len(data)) # first 99% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [6]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

25.413705 M parameters


In [9]:
# create a PyTorch optimizer
start=time.time()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

end=time.time()
print(end-start)

step 0: train loss 4.5103, val loss 4.5200
step 10: train loss 3.2109, val loss 3.2194
step 20: train loss 2.8814, val loss 2.8897
step 30: train loss 2.6717, val loss 2.6543
step 40: train loss 2.5440, val loss 2.5267
step 50: train loss 2.4774, val loss 2.4680
step 60: train loss 2.4470, val loss 2.4260
step 70: train loss 2.4207, val loss 2.4030
step 80: train loss 2.4148, val loss 2.3952
step 90: train loss 2.4027, val loss 2.3826
step 100: train loss 2.3912, val loss 2.3698
step 110: train loss 2.3863, val loss 2.3727
step 120: train loss 2.3839, val loss 2.3694
step 130: train loss 2.3807, val loss 2.3669
step 140: train loss 2.3699, val loss 2.3523
step 150: train loss 2.3646, val loss 2.3483
step 160: train loss 2.3653, val loss 2.3438
step 170: train loss 2.3538, val loss 2.3368
step 180: train loss 2.3506, val loss 2.3406
step 190: train loss 2.3434, val loss 2.3291
step 200: train loss 2.3358, val loss 2.3155
step 210: train loss 2.3247, val loss 2.3057
step 220: train loss 

In [11]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))


हमारी ।
तुम्हारे लब की सुर्ख़ी लअ'ल की में हमारी का 
जिस को कई मंज़िल से होते तो सवाला ।
इश्क़ पर ज़ोर नहीं है ये वो आतिश 'ग़ालिब' 
कि लगाए न लगे और बुझाए न बने ।
करूँगा क्या जो मोहब्बत में हम ने 
आशि


In [10]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))


आदमी मजबूरी को हम-वतन न हो कर 
यही ज़िंदगी तो है इक ख़्वाब से ख़्वाब ।
हर एक बात ज़माने की आँख में भर आँसू 
ये कौन बता देखिए कि ख़ुदा को दुआ से मिला ।
ऐ ग़म-ए-ज़िंदगी न हो नासेह अम्न निकल एक सिखा 
हमा


In [14]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=200)[0].tolist()))


सब तिरे आँसू पे अकेले छोड़ने न जाएँ 
मिट्टी पे निगाहें महीने तू कर ले है ।
ऐसे से ख़ुद को फ़रेब खा के तासूर 
दुश्वार किया तो है ज़रा उन को सदिया ।
ख़ुदा से ज़मीन जब उठा हाथ देख कर 
इक और अकेला हो इलाह


In [12]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
final_shayri=decode(m.generate(context, max_new_tokens=200)[0].tolist())
final_shayri_lined=final_shayri.split('।')
print(final_shayri_lined[0] + '।\n')
print(final_shayri_lined[1] + '।\n')


सजाया-ए-मुनाजार ये कहाँ की ख़ातिर 
बात पे कहाँ बनावे की ।
नश्शा बहुत तो दिन से महशर में उठास होते हैं 
ऐ कुछ शिकवा दिल बे-ख़ुदी किया आप ।
अंगड़ाई ले के छाब मानते हैं कि 'फ़ैज़' 
होते हैं सितारों से ही


In [13]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
final_shayri=decode(m.generate(context, max_new_tokens=200)[0].tolist())
final_shayri_lined=final_shayri.split('।')
print(final_shayri_lined[0] + '।\n')
print(final_shayri_lined[1] + '।\n')


' मजबूरी मुझे तो यक़ीं नहीं पसंद नहीं ।
मज़दूर की तलाश पे भी दबा कफ़न 
लौटता हूँ मज़मूं कि ये क़त्ल कि अब कौन ।
'मीर' जागीर' कल-अमीर' का है 
अभी तक़्क़ाज भी तो कर आदमी ग़ज़ल ।
एक दिन फ़ुर्क़त पे आसानी
